# Код для создания видеосэмплов отдельных балетных па
Видеосэмплы используют для предпросмотра балетных па из Библиотеки балетных па.

Видеосэмплы созданы на основе кадров (фреймов) из Библиотеки балетных па (созданной в процессе выполнения проекта)

Для экономии памяти, видеосэмплы сделаны в уменьшенном формате: **256 х 144** пикселей

## Алгоритм создания видеосэмплов

1. Напишем функцию **get_frames**: для получения списка кадров, из которых состоит балетный элемент
2. Напишем функцию **generate_video**: для генерирования видеосэмплов из списка кадров
3. Импортируем необходимые библиотеки
4. Загружаем excel-файл и формируем два датафрейма: **elements** и **frame_index**
5. Изменяем размер кадров (фреймов): делаем уменьшенные копии (размера 256 на 144)
6. Генерируем видеосэмплы и сохраняем их в соответствующие папки

### 1. Функция для получения списка кадров, из которых состоит балетный элемент

+  создаем функцию get_frames, которая по индентификатору балетного элемента (id_element) выдает список фреймов (кадров), из которых состоит балетный элемент

**id_element** состоит из трех частей: `Элемент_Балет_Num`
+ название элемента
+  название балета (балетного номера)
+ порядковый номер этого элемента в этом балетном номере среди аналогичных элементов

In [93]:
def get_frames(id_element):
    """
    Функция для получения списка кадров (названия jpg файлов), из которых состоит балетный элемент.

    Параметры:
    -----------
    id_element: str
        Название балетного элемента в формате "Элемент_Балет_Номер"

    """
    # определяем название первого кадра, из которого состоит конкретный балетный элемент
    first_frame = elements[elements['id_element'] == id_element]['Begin'].values[0]

    # добавляем к названию тип файла (расширение)
    first_frame_jpg = first_frame + '.jpg'

    # определяем название последнего кадра, из которого состоит конкретный балетный элемент
    last_frame = elements[elements['id_element'] == id_element]['End'].values[0]
    last_frame_jpg = last_frame + ".jpg"

    # индекс первого кадра
    index_begin = frame_index[frame_index['frames']==first_frame_jpg].index[0]

    # индекс последнего кадра
    index_end = frame_index[frame_index['frames']==last_frame_jpg].index[0]

    # формируем список из индексов кадров
    # в список входит индекс первого кадра, последнего кадра и всех промежуточных кадров
    frames_list = list(range(index_begin, index_end))

    # по списку из индексов отбираем названия кадров, которые формируют наш id_element
    frames = frame_index[frame_index.index.isin(list(range(index_begin, index_end+1)))]

    # формируем список из кадров, которые относятся к данному элементу
    pictures = list(frame_index[frame_index.index.isin(list(range(index_begin,index_end)))]['frames'])

    return pictures


#### Разбор функции get_frames

In [94]:
# Берем произвольный id_element и ищем строку в датафрейме elements, в которой он содержится
id_element = "Arabesque_Baryshnikov_1"
id_element

'Arabesque_Baryshnikov_1'

In [95]:
# Строка id_element из датафрейма elements
elements['id_element']

0          A la seconde_Baryshnikov_1
1          A la seconde_Baryshnikov_2
2          A la seconde_Baryshnikov_3
3      A la seconde_Sleeping Beauty_1
4             Arabesque_Baryshnikov_1
                    ...              
403                Фуэте_Black Swan_1
404                Фуэте_Black Swan_2
405                 Фуэте_Fei Draze_1
406                 Фуэте_Fei Draze_2
407                 Фуэте_Fei Draze_3
Name: id_element, Length: 408, dtype: object

In [96]:
# Буллева маска: значение True в той строке, в которой содержится наш id_element
elements['id_element'] == id_element

0      False
1      False
2      False
3      False
4       True
       ...  
403    False
404    False
405    False
406    False
407    False
Name: id_element, Length: 408, dtype: bool

In [97]:
# Применяем буллеву маску к датафрэйму elements, получаем строку, в которой содержится наш id_element
elements[elements['id_element'] == id_element]

,Begin,End,id_element,Элемент,Балет,NN,Num_Frames,Begin_Элемента,End_Элемента
4,Baryshnikov_0-00-05.16,Baryshnikov_0-00-06.20,Arabesque_Baryshnikov_1,Arabesque,Baryshnikov,1,27,Baryshnikov_0-00-05.16,Baryshnikov_0-00-06.20


In [98]:
# Из полученной строки вытаскиваем значение из столбца Begin, это - имя первого кадра нашего балетного элемента
first_frame = elements[elements['id_element'] == id_element]['Begin'].values[0]
first_frame

'Baryshnikov_0-00-05.16'

In [99]:
# К имени кадра добавим расширение .jpg - получим название файла с картинкой
first_frame_jpg = first_frame + '.jpg'
first_frame_jpg

'Baryshnikov_0-00-05.16.jpg'

In [100]:
# Аналогично узнаем название файла с последним кадром для данного балетного элемента
last_frame = elements[elements['id_element'] == id_element]['End'].values[0]
last_frame_jpg = last_frame + ".jpg"
last_frame_jpg

'Baryshnikov_0-00-06.20.jpg'

In [101]:
# переходим к датафрейму frame_index
# в этом датафрейме содержится список всех кадров из всех используемых балетных номеров (44 тысячи кадров) и уникальные индексы - взаимооднозначные
# т.е. каждому индексу соответствует только один кадр 
# и у каждого кадра только один индекс
# и, зная значение индекса, мы можем получить имя соответствующего кадра.
frame_index['frames']

index
0            Baryshnikov_0-00-00.00.jpg
1            Baryshnikov_0-00-00.04.jpg
2            Baryshnikov_0-00-00.08.jpg
3            Baryshnikov_0-00-00.12.jpg
4            Baryshnikov_0-00-00.16.jpg
                      ...              
44211    Tanec Fei Draze_0-03-30.47.jpg
44212    Tanec Fei Draze_0-03-30.50.jpg
44213    Tanec Fei Draze_0-03-30.53.jpg
44214    Tanec Fei Draze_0-03-30.57.jpg
44215    Tanec Fei Draze_0-03-30.60.jpg
Name: frames, Length: 44216, dtype: object

In [102]:
# применяем буллеву маску: сравниваем значение в столбце frames с именем первого кадра балетного элемента
# мы получим везде False, кроме одной ячейки (в которой будет True) - в этой ячейке находится имя первого кадра, нас интересует индекс этой ячейки.
frame_index['frames'] == first_frame_jpg

index
0        False
1        False
2        False
3        False
4        False
         ...  
44211    False
44212    False
44213    False
44214    False
44215    False
Name: frames, Length: 44216, dtype: bool

In [103]:
# применяем буллеву маску и получаем строку с именем первого кадра
frame_index[frame_index['frames'] == first_frame_jpg]

,frames
index,
129,Baryshnikov_0-00-05.16.jpg


In [104]:
# получаем индекс строки с именем первого кадра
index_begin = frame_index[frame_index['frames'] == first_frame_jpg].index[0]
index_begin

129

In [105]:
# аналогично, полуаем индекс для последнего кадра
frame_index[frame_index['frames'] == last_frame_jpg]
index_end = frame_index[frame_index['frames'] == last_frame_jpg].index[0]
index_end

155

In [106]:
# используем генератор списков range для получения индексов промежуточных кадров
# так как генератор списка не использует последнее значение диапазона, то, чтобы не потерять индекс последнего кадра, мы увеличим его на 1
# в итоге получаем список, в котором есть индекс первого кадра, индекс последнего кадра и индексы всех промежуточных кадров
frames_list = list(range(index_begin,index_end+1))
frames_list

[129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155]

In [107]:
# буллева маска - по списку из индексов отбираем названия кадров, которые формируют наш id_element
frame_index.index.isin(list(range(index_begin,index_end)))

array([False, False, False, ..., False, False, False])

In [108]:
# с помощью буллевой маски отбираем те кадры, которые формируют наш балетный элемент
list(frame_index[frame_index.index.isin(list(range(index_begin,index_end)))]['frames'])

['Baryshnikov_0-00-05.16.jpg',
 'Baryshnikov_0-00-05.20.jpg',
 'Baryshnikov_0-00-05.24.jpg',
 'Baryshnikov_0-00-05.28.jpg',
 'Baryshnikov_0-00-05.32.jpg',
 'Baryshnikov_0-00-05.36.jpg',
 'Baryshnikov_0-00-05.40.jpg',
 'Baryshnikov_0-00-05.44.jpg',
 'Baryshnikov_0-00-05.48.jpg',
 'Baryshnikov_0-00-05.52.jpg',
 'Baryshnikov_0-00-05.56.jpg',
 'Baryshnikov_0-00-05.60.jpg',
 'Baryshnikov_0-00-05.64.jpg',
 'Baryshnikov_0-00-05.68.jpg',
 'Baryshnikov_0-00-05.72.jpg',
 'Baryshnikov_0-00-05.76.jpg',
 'Baryshnikov_0-00-05.80.jpg',
 'Baryshnikov_0-00-05.84.jpg',
 'Baryshnikov_0-00-05.88.jpg',
 'Baryshnikov_0-00-05.92.jpg',
 'Baryshnikov_0-00-05.96.jpg',
 'Baryshnikov_0-00-06.00.jpg',
 'Baryshnikov_0-00-06.04.jpg',
 'Baryshnikov_0-00-06.08.jpg',
 'Baryshnikov_0-00-06.12.jpg',
 'Baryshnikov_0-00-06.16.jpg']

### 2. Функция для генерирования видеосэмплов из списка кадров
+ создаем функцию generate_video, которая генерирует видеофайл из набора (списка) фреймов (кадров): для каждого балетного элемента свой видеофайл 

In [109]:
def generate_video(id_element, samples_folder, image_folder):
    """
    Функция для генерирования видеосэмплов из списка кадров.
    По умолчанию формат видео: 256 на 144

    Параметры:
    + id_element: string,
        Идентификатор балетного элемента
    + samples_folder: string
        Папка, в которую сохраняются видеосэмплы
    + image_folder: string
        Папка с кадрами (уменьшенными)

    """
    width=256
    height=144
    print(f'{id_element} sample ')

    # указываем, как будут называться файлы-видеосэмплы
    video_name = os.path.join(samples_folder, f'{id_element}.avi')

    # применяем нашу функцию get_frames
    # получаем список кадров, которые формируют конкретный балетный элемент
    images = get_frames(id_element)

    # создаем экземпляр VideoWriter
    # указываем параметры: название файла, кодек по умолчанию, частота кадров (25),
    video = cv2.VideoWriter(video_name, 0, 25, (width, height))

    # генерируем видео последовательно, добавляя кадры из нашего списка
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Очищаем память
    cv2.destroyAllWindows()
    video.release()



## Создание видео

### 3. Импортируем необходимые библиотеки

In [110]:
import pandas as pd # пандас - библиотека для обработки excel файлов
import cv2 # open cv - библиотека для получения видео из набора кадров (фреймов)
import os # библиотека для работы с именами файлов и их расположением

from PIL import Image # для изменения размера кадров

### 4. Загружаем excel-файл и формируем два датафрейма

In [111]:
# загружаем датафреймы
file = "Create_Ballet_v1.02.xlsx"

elements = pd.read_excel(file, 
                         sheet_name="Элементы_экземпляры_кадры",
                        header=0)

In [112]:
# смотрим первые 5 строк
elements.head()

,Begin,End,id_element,Элемент,Балет,NN,Num_Frames,Begin_Элемента,End_Элемента
0,Baryshnikov_0-00-42.28,Baryshnikov_0-00-43.04,A la seconde_Baryshnikov_1,A la seconde,Baryshnikov,1,20,Baryshnikov_0-00-42.28,Baryshnikov_0-00-43.04
1,Baryshnikov_0-00-46.56,Baryshnikov_0-00-47.08,A la seconde_Baryshnikov_2,A la seconde,Baryshnikov,2,14,Baryshnikov_0-00-46.56,Baryshnikov_0-00-47.08
2,Baryshnikov_0-00-50.52,Baryshnikov_0-00-51.12,A la seconde_Baryshnikov_3,A la seconde,Baryshnikov,3,16,Baryshnikov_0-00-50.52,Baryshnikov_0-00-51.12
3,Sleeping Beauty_0-00-52.12,Sleeping Beauty_0-00-56.52,A la seconde_Sleeping Beauty_1,A la seconde,Sleeping Beauty,1,111,Sleeping Beauty_0-00-52.12,Sleeping Beauty_0-00-56.52
4,Baryshnikov_0-00-05.16,Baryshnikov_0-00-06.20,Arabesque_Baryshnikov_1,Arabesque,Baryshnikov,1,27,Baryshnikov_0-00-05.16,Baryshnikov_0-00-06.20


In [113]:
frame_index = pd.read_excel(file, 
                            sheet_name="кадры_индексы",
                            index_col=0,
                            usecols=['index', 'frames'])
frame_index.head()

,frames
index,
0,Baryshnikov_0-00-00.00.jpg
1,Baryshnikov_0-00-00.04.jpg
2,Baryshnikov_0-00-00.08.jpg
3,Baryshnikov_0-00-00.12.jpg
4,Baryshnikov_0-00-00.16.jpg


### 5. Изменяем размер кадров (фреймов): делаем уменьшенные копии (размера 256 на 144)

In [114]:
folder_frames_big = 'BalletFramesBig'
folder_frames_small = 'BalletFramesSmall'

In [115]:
width = 256
height = 144

### 6. Генерируем видеосэмплы и сохраняем их в соответствующие (дочерние) папки

In [116]:
# Основная (родительская) папка для видеосэмплов
folder_parent = "BalletVideoSamples"

# перебираем все строки (балетные элементы) в датафрейме elements
for idx in elements.index:

    # название дочерних папок - папок для каждой группы элементов
    # эти папки находятся внутри родительской папки "BalletVideoSamples"
    folder_element = os.path.join(folder_parent,
                                  elements.loc[idx, 'Элемент'])

    # проверяем наличие дочерней папки (папки для отдельной группы элементов)
    # если дочерней папки не существует внутри родительской - то создаем её
    if not os.path.isdir(folder_element):
        os.mkdir(folder_element)

    # создаем видеосэмплы с помощью нашей функции generate_video
    # каждый видеосэмпл сохраняем в соответствующую дочернюю папку
    id_element = elements.loc[idx, 'id_element']
    generate_video(id_element, folder_element, folder_frames_small)

print('Готово!')

SyntaxError: incomplete input (1343978595.py, line 5)